In [1]:
import pandas as pd 
import re
import requests
import pickle
import calendar

In [2]:
data = requests.get('https://www.ams.usda.gov/mnreports/gx_gr111.txt').text
data
data2 = data.split('\r\n')
data2

['GX_GR111',
 'Des Moines, IA     Wed, Apr 03, 2019     USDA-IL Market News',
 '',
 'Toledo Grain Report',
 '',
 'Spot bids for grain delivered to Toledo.  The following quotations represent',
 'bids ($/bu) from terminal elevators as of 3:00 p.m. eastern.',
 '',
 '                                    OFF RIVER',
 '',
 'Grain             Cash Bids     Bid Change (¢/bu)     Basis     Basis Change',
 'SRW Wheat       4.6100-4.6600          UP 7        -10K to -5K      UNCH',
 'Corn            3.5175-3.5575        UP 1.25       -11K to -7K      UNCH',
 'Soybeans        8.3875-8.6075        DN 1.25       -60K to -38K     UNCH',
 '',
 '--------------------------------------------------------------------------------',
 '',
 '                                     ON RIVER',
 '',
 'Grain             Cash Bids     Bid Change (¢/bu)     Basis     Basis Change',
 'SRW Wheat       4.6100-4.6600          UP 7        -10K to -5K      UNCH',
 'Corn                3.5575           UP 1.25           -7K  

In [3]:
cols = ['Grain','Cash Bids', 'Bid Change', 'Basis', 'Basis Change']


In [4]:
r = re.compile(r'^Grain')
reports = re.split(r,data) #splits based on crop
reports
reps = [x.split('\r\n') for x in reports]

reps

[['GX_GR111',
  'Des Moines, IA     Wed, Apr 03, 2019     USDA-IL Market News',
  '',
  'Toledo Grain Report',
  '',
  'Spot bids for grain delivered to Toledo.  The following quotations represent',
  'bids ($/bu) from terminal elevators as of 3:00 p.m. eastern.',
  '',
  '                                    OFF RIVER',
  '',
  'Grain             Cash Bids     Bid Change (¢/bu)     Basis     Basis Change',
  'SRW Wheat       4.6100-4.6600          UP 7        -10K to -5K      UNCH',
  'Corn            3.5175-3.5575        UP 1.25       -11K to -7K      UNCH',
  'Soybeans        8.3875-8.6075        DN 1.25       -60K to -38K     UNCH',
  '',
  '--------------------------------------------------------------------------------',
  '',
  '                                     ON RIVER',
  '',
  'Grain             Cash Bids     Bid Change (¢/bu)     Basis     Basis Change',
  'SRW Wheat       4.6100-4.6600          UP 7        -10K to -5K      UNCH',
  'Corn                3.5575           U

In [5]:
list1 = []
for x in reps[:]:
    new = [ y for y in x if (y.startswith('SRW Wheat') or y.startswith('Soybeans') or y.startswith('Corn')) ]
    if new !=[]:
        list1.append(new)
location = ['Off River']*int(len(list1[0])/2)+['On river']*int(len(list1[0])/2)
location

['Off River', 'Off River', 'Off River', 'On river', 'On river', 'On river']

In [6]:



def get_report_months():
    
    time = [pd.Timestamp.now().month, (pd.Timestamp.now() + pd.to_timedelta(1,unit='M')).month, (pd.Timestamp.now() + pd.to_timedelta(4,unit='M')).month]
    time
    return [calendar.month_abbr[x] for x in time]

times = get_report_months()
times
# for x in list1:
#     dates_frame = bid_dates[0]*len(x)
# dates_frame

['Apr', 'May', 'Aug']

In [7]:
list3= []
for x in list1:
    for y in x:
        list3.append(re.split(r'\s{2,}',y)) 
        
df3= pd.DataFrame(list3,columns=cols)
df3['location'] = location
df3['date'] = pd.Timestamp.now()
# df3['bid dates'] =([times[0]]*3+[times[1]]*2+[times[2]]*1)*2
df3

,Grain,Cash Bids,Bid Change,Basis,Basis Change,location,date
0,SRW Wheat,4.6100-4.6600,UP 7,-10K to -5K,UNCH,Off River,2019-04-04 13:05:21.741470
1,Corn,3.5175-3.5575,UP 1.25,-11K to -7K,UNCH,Off River,2019-04-04 13:05:21.741470
2,Soybeans,8.3875-8.6075,DN 1.25,-60K to -38K,UNCH,Off River,2019-04-04 13:05:21.741470
3,SRW Wheat,4.6100-4.6600,UP 7,-10K to -5K,UNCH,On river,2019-04-04 13:05:21.741470
4,Corn,3.5575,UP 1.25,-7K,UNCH,On river,2019-04-04 13:05:21.741470
5,Soybeans,8.4875-8.6075,DN 1.25,-50K to -38K,UNCH,On river,2019-04-04 13:05:21.741470


In [8]:
df3['cash_bids1'] = df3['Cash Bids'].str.split('-', expand=True)[0]
df3['cash_bids2'] = df3['Cash Bids'].str.split('-', expand=True)[1]
df3['basis1'] = df3['Basis'].str.split('to', expand=True)[0].str.replace('[a-zA-Z]','')
df3['basis2'] = df3['Basis'].str.split('to', expand=True)[1].str.replace('[a-zA-Z]','')
df3['contract'] = df3['Basis'].str.split('to', expand=True)[0].str.replace('\d|[+-]',"")

In [9]:
df3

,Grain,Cash Bids,Bid Change,Basis,Basis Change,location,date,cash_bids1,cash_bids2,basis1,basis2,contract
0,SRW Wheat,4.6100-4.6600,UP 7,-10K to -5K,UNCH,Off River,2019-04-04 13:05:21.741470,4.6100,4.6600,-10,-5,K
1,Corn,3.5175-3.5575,UP 1.25,-11K to -7K,UNCH,Off River,2019-04-04 13:05:21.741470,3.5175,3.5575,-11,-7,K
2,Soybeans,8.3875-8.6075,DN 1.25,-60K to -38K,UNCH,Off River,2019-04-04 13:05:21.741470,8.3875,8.6075,-60,-38,K
3,SRW Wheat,4.6100-4.6600,UP 7,-10K to -5K,UNCH,On river,2019-04-04 13:05:21.741470,4.6100,4.6600,-10,-5,K
4,Corn,3.5575,UP 1.25,-7K,UNCH,On river,2019-04-04 13:05:21.741470,3.5575,None,-7,None,K
5,Soybeans,8.4875-8.6075,DN 1.25,-50K to -38K,UNCH,On river,2019-04-04 13:05:21.741470,8.4875,8.6075,-50,-38,K


In [10]:
df3 = df3[['Grain','location','contract','cash_bids1','cash_bids2','Bid Change','basis1','basis2','Basis Change', 'date']]
df3 = df3.fillna('NA')
df3

,Grain,location,contract,cash_bids1,cash_bids2,Bid Change,basis1,basis2,Basis Change,date
0,SRW Wheat,Off River,K,4.6100,4.6600,UP 7,-10,-5,UNCH,2019-04-04 13:05:21.741470
1,Corn,Off River,K,3.5175,3.5575,UP 1.25,-11,-7,UNCH,2019-04-04 13:05:21.741470
2,Soybeans,Off River,K,8.3875,8.6075,DN 1.25,-60,-38,UNCH,2019-04-04 13:05:21.741470
3,SRW Wheat,On river,K,4.6100,4.6600,UP 7,-10,-5,UNCH,2019-04-04 13:05:21.741470
4,Corn,On river,K,3.5575,NA,UP 1.25,-7,NA,UNCH,2019-04-04 13:05:21.741470
5,Soybeans,On river,K,8.4875,8.6075,DN 1.25,-50,-38,UNCH,2019-04-04 13:05:21.741470


In [11]:
pickle.dump(df3, open('gxgr111_afternoonframe','wb'))

In [12]:
df3.to_csv('gxgr111.csv')